# Домашнее задание № 8

## Задание 1 (4 балла) 

Обучите 7 моделей для задачи классификации текста (датасет - lenta_40k ). А именно:  
1) модель с 1 GRU слоем;   
2) модель с 1 LSTM слоем    
3) модель с 1 GRU и 1 LSTM слоем  
4) модель с 1 BIGRU и 2 LSTM слоями  
5) модель с 5 GRU слоями и 3 LSTM слоями  
6) модель 1 BIGRU и 1 BILSTM слоями, причем так чтобы модели для forward и backward прохода отличались   
7) модель, где последовательно идут слои: LSTM, GRU, BILSTM, BIGRU, GRU, LSTM  


Параметр units и размер эмбединга можете задать любой. Оцените качество каждой модели и определите победителя.

In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np
from string import punctuation
from sklearn.model_selection import train_test_split
from collections import Counter
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pyplot as plt
%matplotlib inline

__Возьмем небольшой кусок датасета lenta.ru__

In [2]:
data = pd.read_csv('lenta_40k.csv')

In [3]:
# Предобработка нам тут особо не важна
def preprocess(text):
    tokens = text.lower().split()
    tokens = [token.strip(punctuation) for token in tokens]
    return tokens

In [4]:
# для рассчёта f-меры возьмём функцию из stackoverflow, предложенную на паре 
from tensorflow.keras import backend as K
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [5]:
# создаем словарь, фильтруем, чтобы он был адекватного размера и переводим токены в индексы
vocab = Counter()

for text in data.text:
    vocab.update(preprocess(text))

In [6]:
# отфильтрованный словарь
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 30:
        filtered_vocab.add(word)

In [7]:
# индексируем слова
word2id = {'PAD':0, 'UNK':1}

for word in filtered_vocab:
    word2id[word] = len(word2id)

In [8]:
id2word = {i:word for word, i in word2id.items()}

In [9]:
# переводим тексты в последовательности индексов
X = []

for text in data.text:
    tokens = preprocess(text)
    ids = [word2id.get(token, 1) for token in tokens]
    X.append(ids)

In [10]:
MEAN_LEN = np.median([len(x) for x in X])

In [11]:
MAX_LEN = int(MEAN_LEN + 30)

In [12]:
# паддинг
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=MAX_LEN)

In [13]:
id2label = {i:label for i, label in enumerate(set(data.topic.values))}
label2id = {l:i for i, l in id2label.items()}

In [14]:
y = tf.keras.utils.to_categorical([label2id[label] for label in data.topic.values])

In [15]:
# добавим стратификацию, т.к. в данных у нас дисбаланс классов
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05, stratify=y)

__Создадим требуемые модели__

1) модель с 1 GRU слоем:

In [17]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=30)(inputs, )

gru_layer = tf.keras.layers.GRU(128, return_sequences=False)(embeddings)

outputs = tf.keras.layers.Dense(len(label2id), activation='softmax')(gru_layer)

model_1 = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_1.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[f1, tf.keras.metrics.RecallAtPrecision(0.8, name='rec@prec')])

In [ ]:
model_1.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=1000,
          epochs=20)

In [19]:
model_1_score = model_1.history.history['f1'][-1]
print("f-score модели с 1 GRU слоем:", model_1_score)

f-score модели с 1 GRU слоем: 0.9356775283813477


2) модель с 1 LSTM слоем

In [20]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=30)(inputs, )

lstm_layer = tf.keras.layers.LSTM(128, return_sequences=False)(embeddings)

outputs = tf.keras.layers.Dense(len(label2id), activation='softmax')(lstm_layer)

model_2 = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_2.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[f1, tf.keras.metrics.RecallAtPrecision(0.8, name='rec@prec')])

In [ ]:
model_2.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=1000,
          epochs=20)

In [23]:
model_2_score = model_2.history.history['f1'][-1]
print("f-score модели с 1 LSTM слоем:", model_2_score)

f-score модели с 1 LSTM слоем: 0.9011369943618774


3) модель с 1 GRU и 1 LSTM слоем:

In [16]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=30)(inputs, )

lstm_1 = tf.keras.layers.LSTM(128, return_sequences=True)(embeddings)
lstm_2 = tf.keras.layers.GRU(128, return_sequences=False)(lstm_1)

dense = tf.keras.layers.Dense(64, activation='relu')(lstm_2)
outputs = tf.keras.layers.Dense(len(label2id), activation='softmax')(dense)

model_3 = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_3.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[f1, tf.keras.metrics.RecallAtPrecision(0.8, name='rec@prec')])

In [ ]:
model_3.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=1000,
          epochs=20)

In [18]:
model_3_score = model_3.history.history['f1'][-1]
print("f-score модели с 1 GRU и 1 LSTM слоем:", model_3_score)

f-score модели с 1 GRU и 1 LSTM слоем: 0.9521448612213135


4) модель с 1 BIGRU и 2 LSTM слоями:

In [19]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=30)(inputs, )

bigru_layer = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True))(embeddings)
lstm_layer_1 = tf.keras.layers.LSTM(128, return_sequences=True)(bigru_layer)
lstm_layer_2 = tf.keras.layers.LSTM(128, return_sequences=False)(lstm_layer_1)

dense = tf.keras.layers.Dense(64, activation='relu')(lstm_layer_2)
outputs = tf.keras.layers.Dense(len(label2id), activation='softmax')(dense)

model_4 = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_4.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[f1, tf.keras.metrics.RecallAtPrecision(0.8, name='rec@prec')])

In [ ]:
model_4.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=1000,
          epochs=20)

In [21]:
model_4_score = model_4.history.history['f1'][-1]
print("f-score модели с 1 BIGRU и 2 LSTM слоями:", model_4_score)

f-score модели с 1 BIGRU и 2 LSTM слоями: 0.8436391353607178


5) модель с 5 GRU слоями и 3 LSTM слоями:

In [22]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=30)(inputs, )

gru_1 = tf.keras.layers.GRU(128, return_sequences=True)(embeddings)
gru_2 = tf.keras.layers.GRU(128, return_sequences=True)(gru_1)
gru_3 = tf.keras.layers.GRU(128, return_sequences=True)(gru_2)
gru_4 = tf.keras.layers.GRU(128, return_sequences=True)(gru_3)
gru_5 = tf.keras.layers.GRU(128, return_sequences=True)(gru_4)

lstm_1 = tf.keras.layers.LSTM(128, return_sequences=True)(gru_5)
lstm_2 = tf.keras.layers.LSTM(128, return_sequences=True)(lstm_1)
lstm_3 = tf.keras.layers.LSTM(128, return_sequences=False)(lstm_2)

dense = tf.keras.layers.Dense(64, activation='relu')(lstm_3)
outputs = tf.keras.layers.Dense(len(label2id), activation='softmax')(dense)

model_5 = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_5.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[f1, tf.keras.metrics.RecallAtPrecision(0.8, name='rec@prec')])

In [ ]:
model_5.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=1000,
          epochs=20)

In [1]:
model_5_score = model_5.history.history['f1'][-1]
print("f-score модели с 5 BIGRU и 3 LSTM слоями:", model_5_score)

f-score модели с 5 BIGRU и 3 LSTM слоями: 0.0


6) модель с 1 BIGRU и 1 BILSTM слоями, причем так чтобы модели для forward и backward прохода отличались:

In [ ]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=30)(inputs, )

bigru_bilstm = tf.keras.layers.Bidirectional(
                                       tf.keras.layers.GRU(128, return_sequences=False),
                        backward_layer=tf.keras.layers.LSTM(128, return_sequences=False, 
                                                            go_backwards=True))(embeddings)

outputs = tf.keras.layers.Dense(len(label2id), activation='softmax')(bigru_bilstm)
model_6 = tf.keras.Model(inputs=inputs, outputs=outputs)

model_6.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=[f1, tf.keras.metrics.RecallAtPrecision(0.8, name='rec@prec')])

In [ ]:
model_6.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=1000,
          epochs=20)

In [2]:
model_6_score = model_6.history.history['f1'][-1]
print("f-score модели с 1 BIGRU и 1 BILSTM слоями:", model_6_score)

f-score модели с 1 BIGRU и 1 BILSTM слоями: 0.8516635623931885


7) модель, где последовательно идут слои: LSTM, GRU, BILSTM, BIGRU, GRU, LSTM:

In [ ]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=30)(inputs, )

lstm_1 = tf.keras.layers.LSTM(128, return_sequences=True)(embeddings)
gru_1 = tf.keras.layers.GRU(128, return_sequences=True)(lstm_1)
bilstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(gru_1)
bigru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True))(bilstm)
gru_2 = tf.keras.layers.GRU(128, return_sequences=True)(bigru)
lstm_2_final = tf.keras.layers.LSTM(128, return_sequences=False)(gru_2)

dense = tf.keras.layers.Dense(64, activation='relu')(lstm_2_final)
outputs = tf.keras.layers.Dense(len(label2id), activation='softmax')(dense)

model_7 = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model_7.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[f1, tf.keras.metrics.RecallAtPrecision(0.8, name='rec@prec')])

In [ ]:
model_7.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=1000,
          epochs=20)

In [3]:
model_7_score = model_7.history.history['f1'][-1]
print("f-score модели, где последовательно идут слои: LSTM, GRU, BILSTM, BIGRU, GRU, LSTM:", model_7_score)

f-score модели, где последовательно идут слои: LSTM, GRU, BILSTM, BIGRU, GRU, LSTM: 0.6749071973800659


__Лучше всего себя показала модель с  с 1 GRU и 1 LSTM слоем__

## Задание 2 (6 баллов)


На данных википедии (wikiann) обучите 2 модели:  
1) модель в которой будут использованы предобученные эмбединги слов и несколько BILSTM слоев. 
1) модель в которой будут использованы предобученные эмбединги слов и несколько BIGRU слоев. 

Сравните качество по метрикам. Также придумайте несколько сложных примеров и проверьте, какие сущности определяет каждая из моделей.

In [21]:
from sklearn.metrics import classification_report

In [5]:
from datasets import load_dataset

In [6]:
wikiann_dataset = load_dataset("wikiann", 'ru')

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Dataset wikiann downloaded and prepared to C:\Users\tanbe\.cache\huggingface\datasets\wikiann\ru\1.1.0\4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
# словарь
vocab = Counter()

for sent in wikiann_dataset['train']['tokens']:
    vocab.update([x.lower() for x in sent])

In [9]:
# индексируем слова
word2id = {'PAD':0, 'UNK':1}

for word in vocab:
    word2id[word] = len(word2id)

In [10]:
id2word = {i:word for word, i in word2id.items()}

In [11]:
# переводим тексты в последовательности индексов
X = []

for sent in wikiann_dataset['train']['tokens']:
    tokens = [w.lower() for w in sent]
    ids = [word2id.get(token, 1) for token in tokens]
    X.append(ids)

In [12]:
# переводим тексты в последовательности индексов
X_test = []

for sent in wikiann_dataset['test']['tokens']:
    tokens = [w.lower() for w in sent]
    ids = [word2id.get(token, 1) for token in tokens]
    X_test.append(ids)

In [13]:
MAX_LEN = max(len(x) for x in X)

# паддинг
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=MAX_LEN, padding='post')
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=MAX_LEN, padding='post')

In [14]:
id2labels = {0:'O', 1:'B-PER', 2:'I-PER', 3:'B-ORG', 4:'I-ORG', 5: 'B-LOC', 6:'I-LOC', 7:'PAD'}
label2id = {v:k for k,v in id2labels.items()} 

In [16]:
y = tf.keras.preprocessing.sequence.pad_sequences(wikiann_dataset['train']['ner_tags'], value=7,
                                                  maxlen=MAX_LEN,  padding='post')
y_test = tf.keras.preprocessing.sequence.pad_sequences(wikiann_dataset['test']['ner_tags'], value=7,
                                                       maxlen=MAX_LEN,  padding='post')

Модель, в которой будут использованы предобученные эмбединги слов и несколько BILSTM слоев. Я взяла 3 слоя.

In [17]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=100)(inputs)

lstm_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(embeddings)
lstm_2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(lstm_1)
lstm_3 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(lstm_2)

outputs = tf.keras.layers.Dense(len(label2id), activation='softmax')(lstm_3)

bilstm_model = tf.keras.Model(inputs=inputs, outputs=outputs)
bilstm_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', 
             metrics=['accuracy'])

In [ ]:
bilstm_model.fit(X, y, 
          validation_data=(X_test, y_test),
          batch_size=128,
         epochs=5)

In [19]:
pred = bilstm_model.predict(X_test).argmax(2)

In [22]:
print(classification_report(y_test.reshape(-1), pred.reshape(-1), labels=list(id2labels.keys()),
                                                                     target_names=list(id2labels.values()),
                                                                     zero_division=0))

              precision    recall  f1-score   support

           O       0.93      0.95      0.94     40480
       B-PER       0.81      0.82      0.81      3542
       I-PER       0.94      0.87      0.90      7544
       B-ORG       0.77      0.59      0.67      4074
       I-ORG       0.88      0.72      0.79      8008
       B-LOC       0.60      0.78      0.68      4560
       I-LOC       0.60      0.76      0.67      3060
         PAD       1.00      1.00      1.00    468732

    accuracy                           0.98    540000
   macro avg       0.82      0.81      0.81    540000
weighted avg       0.98      0.98      0.98    540000



Модель, в которой будут использованы предобученные эмбединги слов и несколько BIGRU слоев. Я взяла 3.

In [23]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=100)(inputs)

gru_1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True))(embeddings)
gru_2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True))(gru_1)
gru_3 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, return_sequences=True))(gru_2)

outputs = tf.keras.layers.Dense(len(label2id), activation='softmax')(gru_3)

bigru_model = tf.keras.Model(inputs=inputs, outputs=outputs)
bigru_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', 
             metrics=['accuracy'])

In [ ]:
bigru_model.fit(X, y, 
          validation_data=(X_test, y_test),
          batch_size=128,
          epochs=5)

In [25]:
pred = bigru_model.predict(X_test).argmax(2)

In [26]:
print(classification_report(y_test.reshape(-1), pred.reshape(-1), labels=list(id2labels.keys()),
                                                                     target_names=list(id2labels.values()),
                                                                     zero_division=0))

              precision    recall  f1-score   support

           O       0.97      0.89      0.93     40480
       B-PER       0.80      0.89      0.84      3542
       I-PER       0.90      0.92      0.91      7544
       B-ORG       0.71      0.69      0.70      4074
       I-ORG       0.86      0.78      0.81      8008
       B-LOC       0.55      0.87      0.68      4560
       I-LOC       0.63      0.82      0.71      3060
         PAD       1.00      1.00      1.00    468732

    accuracy                           0.98    540000
   macro avg       0.80      0.86      0.82    540000
weighted avg       0.98      0.98      0.98    540000



In [27]:
import re

def tokenize(text, word2id):
    # токенизирует и переводит в индексы
    tokens = re.findall('\w+|[^\w\s]+', text)
    ids = [word2id.get(token.lower(), 1) for token in tokens]
    return tokens, ids

def pred2tags(pred, id2label, length):
    # декодирует индексы в части речи
    # length нужно чтобы откидывать паддинги или некорректные предсказания
    pred = pred.argmax(2)[0, :length]
    labels = [id2label[l] for l in pred]
    return labels

def label_seq(text, word2id, id2label, max_len, model):
    tokens, ids = tokenize(text, word2id)
    pred = model.predict(tf.keras.preprocessing.sequence.pad_sequences([ids], 
                                                                       maxlen=max_len, 
                                                                       padding='post'))
    labels = pred2tags(pred, id2label, len(ids))
    
    return list(zip(tokens, labels))

In [28]:
label_seq('В белом плаще с кровавым подбоем ранним утром четырнадцатого числа весеннего месяца нисана в крытую колоннаду между двумя крыльями дворца ирода великого вышел прокуратор Иудеи Понтий Пилат.', word2id, id2labels, MAX_LEN, bilstm_model)

[('В', 'O'),
 ('белом', 'O'),
 ('плаще', 'O'),
 ('с', 'O'),
 ('кровавым', 'O'),
 ('подбоем', 'O'),
 ('ранним', 'O'),
 ('утром', 'O'),
 ('четырнадцатого', 'O'),
 ('числа', 'O'),
 ('весеннего', 'O'),
 ('месяца', 'O'),
 ('нисана', 'O'),
 ('в', 'O'),
 ('крытую', 'O'),
 ('колоннаду', 'O'),
 ('между', 'O'),
 ('двумя', 'O'),
 ('крыльями', 'O'),
 ('дворца', 'B-PER'),
 ('ирода', 'I-PER'),
 ('великого', 'I-PER'),
 ('вышел', 'O'),
 ('прокуратор', 'O'),
 ('Иудеи', 'I-PER'),
 ('Понтий', 'O'),
 ('Пилат', 'O'),
 ('.', 'O')]

In [29]:
label_seq('В белом плаще с кровавым подбоем ранним утром четырнадцатого числа весеннего месяца нисана в крытую колоннаду между двумя крыльями дворца ирода великого вышел прокуратор Иудеи Понтий Пилат.', word2id, id2labels, MAX_LEN, bigru_model)

[('В', 'O'),
 ('белом', 'B-ORG'),
 ('плаще', 'I-ORG'),
 ('с', 'O'),
 ('кровавым', 'O'),
 ('подбоем', 'I-ORG'),
 ('ранним', 'I-ORG'),
 ('утром', 'I-ORG'),
 ('четырнадцатого', 'I-ORG'),
 ('числа', 'O'),
 ('весеннего', 'O'),
 ('месяца', 'O'),
 ('нисана', 'O'),
 ('в', 'O'),
 ('крытую', 'B-LOC'),
 ('колоннаду', 'O'),
 ('между', 'O'),
 ('двумя', 'O'),
 ('крыльями', 'O'),
 ('дворца', 'B-PER'),
 ('ирода', 'I-LOC'),
 ('великого', 'B-PER'),
 ('вышел', 'O'),
 ('прокуратор', 'B-PER'),
 ('Иудеи', 'B-PER'),
 ('Понтий', 'I-PER'),
 ('Пилат', 'I-PER'),
 ('.', 'O')]

In [30]:
label_seq('От другого этого места у Никанора Ивановича осталось в воспоминании мало чего.', word2id, id2labels, MAX_LEN, bilstm_model)

[('От', 'O'),
 ('другого', 'O'),
 ('этого', 'O'),
 ('места', 'O'),
 ('у', 'O'),
 ('Никанора', 'B-LOC'),
 ('Ивановича', 'O'),
 ('осталось', 'O'),
 ('в', 'O'),
 ('воспоминании', 'O'),
 ('мало', 'O'),
 ('чего', 'O'),
 ('.', 'O')]

In [31]:
label_seq('От другого этого места у Никанора Ивановича осталось в воспоминании мало чего.', word2id, id2labels, MAX_LEN, bigru_model)

[('От', 'O'),
 ('другого', 'O'),
 ('этого', 'O'),
 ('места', 'O'),
 ('у', 'O'),
 ('Никанора', 'B-LOC'),
 ('Ивановича', 'O'),
 ('осталось', 'O'),
 ('в', 'O'),
 ('воспоминании', 'B-LOC'),
 ('мало', 'O'),
 ('чего', 'O'),
 ('.', 'O')]

In [32]:
label_seq('Маргарита подняла голову к луне и сделала задумчивое и поэтическое лицо.', word2id, id2labels, MAX_LEN, bilstm_model)

[('Маргарита', 'B-PER'),
 ('подняла', 'I-PER'),
 ('голову', 'O'),
 ('к', 'O'),
 ('луне', 'B-PER'),
 ('и', 'O'),
 ('сделала', 'O'),
 ('задумчивое', 'I-PER'),
 ('и', 'O'),
 ('поэтическое', 'O'),
 ('лицо', 'O'),
 ('.', 'O')]

In [33]:
label_seq('Маргарита подняла голову к луне и сделала задумчивое и поэтическое лицо.', word2id, id2labels, MAX_LEN, bigru_model)

[('Маргарита', 'B-LOC'),
 ('подняла', 'I-LOC'),
 ('голову', 'I-LOC'),
 ('к', 'I-LOC'),
 ('луне', 'I-LOC'),
 ('и', 'O'),
 ('сделала', 'O'),
 ('задумчивое', 'B-LOC'),
 ('и', 'O'),
 ('поэтическое', 'B-LOC'),
 ('лицо', 'O'),
 ('.', 'O')]

__И BIGRU и BILSTM модели ошибаются при определении сущностей, но BIGRU немного лучше.__
__По метрикам - результаты примерно одинаковые, но у BIGRU результаты повыше.__